# Lab3 (Student version)

In [4]:
import random
import time
import matplotlib.pyplot as plt
import sys
import timeit
from collections import defaultdict
import queue
from collections import deque
import multiprocessing as mp

Download the three following graphs on Moodle:
- IP-inet-undir.txt (small map of the internet at the IP level obtained with traceroute measurements, source not communicated)
- AS-Caida-undir.txt (snapshot map of the internet at the AS level in 2007 obtained using BGP tables, part of project CAIDA)
- IP-Skitter-undir.txt (**careful**: larger map of the internet at the IP level in 2005 obtained with traceroute measurements, part of project CAIDA)

It is also useful to consider some toy graphs (e.g. manually created graphs with a dozen nodes) to test your programs.

## Exercise 0: preliminaries

Using the codes of Lab1, load the graphs in memory as dictionary of lists and check their number of nodes and links.

In [5]:
def load_graph_as_adj_list(path):
    graph = {}

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            u, v = map(int, line.split())

            graph.setdefault(u, []).append(v)
            graph.setdefault(v, []).append(u)

    return graph

In [6]:
def count_graph_from_memory(graph):
    num_nodes = len(graph)
    num_edges = sum(len(neighbors) for neighbors in graph.values()) // 2
    return num_nodes, num_edges

In [7]:
graphs = {
    "IP-inet": "IP-inet-undir.txt",
    "AS-Caida": "AS-Caida-undir.txt",
    "IP-Skitter": "IP-Skitter-undir.txt",
}

for name, path in graphs.items():
    G = load_graph_as_adj_list(path)
    n, m = count_graph_from_memory(G)
    print(f"{name}: {n} nodes, {m} edges")
    
    degrees = [len(neighs) for neighs in G.values()]
    print("Min degree:", min(degrees))
    print("Max degree:", max(degrees))
    print("Avg degree:", sum(degrees)/len(degrees))



IP-inet: 9740 nodes, 35567 edges
Min degree: 1
Max degree: 58
Avg degree: 7.303285420944558
AS-Caida: 26475 nodes, 53381 edges
Min degree: 1
Max degree: 2628
Avg degree: 4.0325590179414545
IP-Skitter: 1696415 nodes, 11095298 edges
Min degree: 1
Max degree: 35455
Avg degree: 13.080877025963575


## Exercise 1: BFS

### 1.1 Components

- Implement a BFS algorithm.  

- Use it on each of the graphs to evaluate the size of the largest connected component (LCC) of these graphs, compute the fraction of nodes in the LCC for each graph.
- Use it to identify all connected components.

Warning: if your BFS is not well coded, it can be very long, so if it doesn't work on Skitter in less than a few minutes, either improve your code, or test only on smaller graphs. 

In [8]:
from collections import deque

def bfs(graph, start):
    visited = set([start])
    q = deque([start])

    while q:
        u = q.popleft()
        for v in graph[u]:
            if v not in visited:
                visited.add(v)
                q.append(v)

    return visited

In [9]:
def connected_components(graph):
    visited_global = set()
    components = []

    for node in graph:
        if node not in visited_global:
            comp = bfs(graph, node)
            components.append(comp)
            visited_global |= comp  # union

    return components


In [10]:
def largest_connected_component(graph):
    comps = connected_components(graph)
    sizes = [len(c) for c in comps]
    max_size = max(sizes)
    lcc = comps[sizes.index(max_size)]
    return lcc, sizes

In [11]:
for name, path in graphs.items():
    print(f"\n--- {name} ---")
    G = load_graph_as_adj_list(path)

    lcc, sizes = largest_connected_component(G)
    frac = len(lcc) / len(G)

    print(f"Nodes: {len(G)}")
    print(f"Connected components: {len(sizes)}")
    print(f"LCC size: {len(lcc)}")
    print(f"Fraction in LCC: {frac:.4f}")



--- IP-inet ---
Nodes: 9740
Connected components: 156
LCC size: 8557
Fraction in LCC: 0.8785

--- AS-Caida ---
Nodes: 26475
Connected components: 1
LCC size: 26475
Fraction in LCC: 1.0000

--- IP-Skitter ---
Nodes: 1696415
Connected components: 756
LCC size: 1694616
Fraction in LCC: 0.9989


### 1.2 Distances

- Modify the BFS above to have it compute the distance to the source node.

- Using the fact that the diameter is necessarily larger than any distance measured, use your distance computation code to get a lower bound of the diameter. The higher the bound, the better.

In [12]:
def bfs_distances(graph, start):
    dist = {start: 0}
    q = deque([start])

    while q:
        u = q.popleft()
        d = dist[u]

        for v in graph[u]:
            if v not in dist:
                dist[v] = d + 1
                q.append(v)

    return dist

def eccentricity(graph, start):
    dist = bfs_distances(graph, start)
    return max(dist.values())

def highest_degree_node(graph):
    return max(graph, key=lambda u: len(graph[u]))

def diameter_lower_bound_top(graph):
    # start with the highest-degree node (good guess)
    start = highest_degree_node(graph)
    dist1 = bfs_distances(graph, start)
    nodeA = max(dist1, key=dist1.get)
    eccA = dist1[nodeA]

    # now BFS from nodeA
    dist2 = bfs_distances(graph, nodeA)
    nodeB = max(dist2, key=dist2.get)
    eccB = dist2[nodeB]

    return max(eccA, eccB)


In [13]:
for name, path in graphs.items():
    print(f"\n--- {name} ---")
    G = load_graph_as_adj_list(path)
    lb = diameter_lower_bound_top(G)
    print(f"Diameter lower bound: {lb}")



--- IP-inet ---
Diameter lower bound: 34

--- AS-Caida ---
Diameter lower bound: 17

--- IP-Skitter ---
Diameter lower bound: 31


## Exercise 2: Triangles

### 2.1 Raw triangle counting

- Implement the 2 triangle counting algorithms presented in the course. 

- Test your program on the 3 graphs and report the number of triangles as well as the running time of your program.

In [14]:
def build_forward_graph(graph):
    # 1. compute degrees
    deg = {u: len(neighs) for u, neighs in graph.items()}
    
    # 2. define node ordering: increasing degree, tie-break on ID
    order = sorted(graph.keys(), key=lambda u: (deg[u], u))
    
    # 3. map node -> rank
    rank = {u: i for i, u in enumerate(order)}
    
    # 4. build forward graph H(u): only neighbors with higher rank
    H = {u: [] for u in graph}
    
    for u in graph:
        ru = rank[u]
        for v in graph[u]:
            if ru < rank[v]:
                H[u].append(v)
        
        H[u].sort()   # sorted lists for fast intersection
    
    return H, order

def count_triangles_degree_ordered(graph):
    H, order = build_forward_graph(graph)
    triangles = 0
    
    for u in order:
        Hu = H[u]
        for v in Hu:
            # intersect H[u] and H[v]
            Hv = H[v]
            i = j = 0
            while i < len(Hu) and j < len(Hv):
                if Hu[i] == Hv[j]:
                    triangles += 1
                    i += 1
                    j += 1
                elif Hu[i] < Hv[j]:
                    i += 1
                else:
                    j += 1
    
    return triangles


In [15]:
def count_triangles_naive(graph):
    triangles = 0

    # Convert adjacency lists to sets for O(1) membership tests
    adj = {u: set(neighs) for u, neighs in graph.items()}

    for v in adj:
        neighbors = list(adj[v])
        d = len(neighbors)

        # examine all pairs (u1, u2)
        for i in range(d):
            u1 = neighbors[i]
            for j in range(i + 1, d):
                u2 = neighbors[j]

                # check if u1 and u2 are connected
                if u2 in adj[u1]:
                    triangles += 1

    return triangles // 3  # naive counts each triangle 3 times


In [16]:
def count_triangles_improved(graph):
    # sorted adjacency lists for fast merging
    adj = {u: sorted(set(neighs)) for u, neighs in graph.items()}
    
    triangles = 0

    for u in adj:
        for v in adj[u]:
            if u < v:   # enforce ordering (u,v)
                # merge-like intersection of neighbors
                nu = adj[u]
                nv = adj[v]
                i = j = 0

                while i < len(nu) and j < len(nv):
                    if nu[i] == nv[j]:
                        w = nu[i]
                        if v < w:    # enforce v < w to count only once
                            triangles += 1
                        i += 1
                        j += 1
                    elif nu[i] < nv[j]:
                        i += 1
                    else:
                        j += 1

    return triangles

In [17]:
def run_all_triangle_counts(graph, name):
    print(f"\n--- {name} ---")

    # Naive + improved ONLY for small graphs
    if name != "IP-Skitter":
        start = time.time()
        t1 = count_triangles_naive(graph)
        end = time.time()
        print("Naive triangles:", t1, "time:", end - start)

        start = time.time()
        t2 = count_triangles_improved(graph)
        end = time.time()
        print("Improved triangles:", t2, "time:", end - start)

    # Degree-ordered (fastest) for ALL graphs
    start = time.time()
    t3 = count_triangles_degree_ordered(graph)
    end = time.time()
    print("Degree-ordered triangles:", t3, "time:", end - start)

for name, path in graphs.items():
    G = load_graph_as_adj_list(path)
    run_all_triangle_counts(G, name)





--- IP-inet ---
Naive triangles: 206909 time: 0.08873224258422852
Improved triangles: 206909 time: 0.12350869178771973
Degree-ordered triangles: 206909 time: 0.0730886459350586

--- AS-Caida ---
Naive triangles: 36365 time: 1.1997900009155273
Improved triangles: 36365 time: 1.1836650371551514
Degree-ordered triangles: 36365 time: 0.1350092887878418

--- IP-Skitter ---
Degree-ordered triangles: 28769868 time: 102.7787024974823


### 2.2 Transitive ratio

Use this program to compute the transitive ratio of the graphs. Remember that the transitive ratio is defined as 
$$ \frac{3.number \ of \ triangles}{number \ of \ forks}$$
and that the number of forks (or connected triples) of a node of degree $d$ is simply $\frac{d(d-1)}{2}$.

In [ ]:
def triangles_by_node(graph):

    H, order = build_forward_graph(graph)

    tri = {u: 0 for u in graph}

    # For fast intersection: H[u] is already sorted
    for u in H:
        Hu = H[u]
        for v in Hu:
            Hv = H[v]
            i = j = 0
            while i < len(Hu) and j < len(Hv):
                if Hu[i] == Hv[j]:
                    w = Hu[i]
                    tri[u] += 1
                    tri[v] += 1
                    tri[w] += 1
                    i += 1
                    j += 1
                elif Hu[i] < Hv[j]:
                    i += 1
                else:
                    j += 1

    return tri

In [ ]:
def transitive_ratio(graph):
    tri = triangles_by_node(graph)

    local = {}
    total_triples = 0

    for u in graph:
        d = len(graph[u])
        max_triples = d * (d - 1) / 2
        if max_triples > 0:
            local[u] = tri[u] / max_triples
            total_triples += max_triples
        else:
            local[u] = 0.0

    total_triangles = sum(tri.values()) / 3
    global_tr = (3 * total_triangles) / total_triples if total_triples > 0 else 0

    return local, global_tr



In [26]:
def run_transitive_ratio(graph, name):
    print(f"\n--- {name} ---")
    local, global_tr = transitive_ratio(graph)

    print(f"Global transitive ratio: {global_tr:.6f}")

    # Show a few nodes for sanity
    print("Sample local TR values:")
    for i, (node, val) in enumerate(local.items()):
        if i == 5: break
        print(f"  node {node}: TR={val:.6f}")


In [27]:
for name, path in graphs.items():
    G = load_graph_as_adj_list(path)
    run_transitive_ratio(G, name)



--- IP-inet ---
Global transitive ratio: 0.877108
Sample local TR values:
  node 0: TR=0.500000
  node 1882: TR=0.666667
  node 7203: TR=0.142857
  node 5853: TR=0.236842
  node 6239: TR=0.047619

--- AS-Caida ---
Global transitive ratio: 0.007319
Sample local TR values:
  node 1: TR=0.000000
  node 2: TR=0.005302
  node 3: TR=0.000000
  node 4: TR=0.002205
  node 30: TR=0.008230

--- IP-Skitter ---
Global transitive ratio: 0.005387
Sample local TR values:
  node 0: TR=0.000248
  node 1: TR=0.001843
  node 2: TR=0.222222
  node 3: TR=0.189474
  node 4: TR=0.200000


## Exercice 3: going further on triangle counting*

This exercise is targeting students interested in more research-oriented work. 

Take a look at the recent article (published in [Alenex 2023](https://epubs.siam.org/doi/book/10.1137/1.9781611977561)
) : https://epubs.siam.org/doi/pdf/10.1137/1.9781611977561.ch7 and implement the algorithms A++ and A+- and compare their running times on the 3 graphs.

**You don't need to understand the whole content of the paper but just implementing given algorithms. Do not hesitate to contact me or email me if you need any help.**

In [ ]:
WARNING THIS TAKES FOREVER TO RUN DONT DO IT 

In [34]:
def degree_ordering(G):
    return sorted(G.keys(), key=lambda u: (len(G[u]), u))

In [35]:
def core_ordering(G):
    # Compute core numbers
    deg = {u: len(G[u]) for u in G}
    core = deg.copy()
    Q = deque([u for u in G if deg[u] == 0])

    while Q:
        u = Q.popleft()
        for v in G[u]:
            if core[v] > core[u]:
                core[v] = core[u]
                Q.append(v)

    # Sort by core number, then ID
    return sorted(G.keys(), key=lambda u: (core[u], u))

In [36]:
def orient_graph(G, ordering):
    rank = {u: i for i, u in enumerate(ordering)}
    succ = {u: [] for u in G}
    pred = {u: [] for u in G}

    for u in G:
        ru = rank[u]
        for v in G[u]:
            if rank[v] > ru:
                succ[u].append(v)
                pred[v].append(u)
    
    # sort to allow two-pointer intersections
    for u in G:
        succ[u].sort()
        pred[u].sort()
    
    return succ, pred, rank


In [37]:
def triangles_App(succ):

    count = 0

    for u in succ:
        Nu = succ[u]
        for v in Nu:
            Nv = succ[v]
            # two-pointer intersection of Nu and Nv
            i = j = 0
            while i < len(Nu) and j < len(Nv):
                if Nu[i] == Nv[j]:
                    count += 1
                    i += 1
                    j += 1
                elif Nu[i] < Nv[j]:
                    i += 1
                else:
                    j += 1

    return count


In [38]:
def triangles_Apm(succ, pred):

    count = 0

    for u in succ:
        for v in succ[u]:  # edge u → v
            Nu = succ[u]
            Pv = pred[v]

            # two-pointer intersection of Nu & Pv
            i = j = 0
            while i < len(Nu) and j < len(Pv):
                if Nu[i] == Pv[j]:
                    count += 1
                    i += 1
                    j += 1
                elif Nu[i] < Pv[j]:
                    i += 1
                else:
                    j += 1

    return count


In [39]:
def run_triangle_algo(G, ordering_fn, algo_name):
    ordering = ordering_fn(G)
    t0 = time.time()

    succ, pred, rank = orient_graph(G, ordering)

    if algo_name == "A++":
        t1 = time.time()
        tri = triangles_App(succ)
    elif algo_name == "A+-":
        t1 = time.time()
        tri = triangles_Apm(succ, pred)
    else:
        raise ValueError("Unknown algorithm")

    t2 = time.time()

    print(f"\nOrdering: {ordering_fn.__name__}  | Algorithm: {algo_name}")
    print(f"Orientation time: {t1 - t0:.4f}s")
    print(f"Listing time: {t2 - t1:.4f}s")
    print(f"Total triangles: {tri}")

In [41]:
for name, path in graphs.items():
    print(f"\n===== {name} =====")
    G = load_graph_as_adj_list(path)

    # Degree ordering
    run_triangle_algo(G, degree_ordering, "A++")
    run_triangle_algo(G, degree_ordering, "A+-")

    # Core ordering
    #run_triangle_algo(G, core_ordering, "A++")
    #run_triangle_algo(G, core_ordering, "A+-")



===== IP-inet =====

Ordering: degree_ordering  | Algorithm: A++
Orientation time: 0.0182s
Listing time: 0.0510s
Total triangles: 206909

Ordering: degree_ordering  | Algorithm: A+-
Orientation time: 0.0159s
Listing time: 0.0900s
Total triangles: 206909

===== AS-Caida =====

Ordering: degree_ordering  | Algorithm: A++
Orientation time: 0.0917s
Listing time: 0.0708s
Total triangles: 36365

Ordering: degree_ordering  | Algorithm: A+-
Orientation time: 0.0493s
Listing time: 0.4521s
Total triangles: 36365

===== IP-Skitter =====

Ordering: degree_ordering  | Algorithm: A++
Orientation time: 17.3610s
Listing time: 6130.3558s
Total triangles: 28769868

Ordering: degree_ordering  | Algorithm: A+-
Orientation time: 22.3255s
Listing time: 4131.6125s
Total triangles: 28769868
